# Advanced Geospatial ML / GIS Template – Spatial Autocorrelation & Kriging (Skeleton)

This template assumes you want to go **beyond tabular** and use real **GIS tools**:

- `geopandas` – vector data (points, polygons)
- `shapely` – geometry operations
- `libpysal` / `esda` – spatial weights, Moran's I, local indicators (LISA)
- `pykrige` – kriging (spatial interpolation)

It is **designed as a skeleton**:

- Not all blocks need to be used at once
- You adapt paths and columns to your data
- Install the required libraries in your environment first


## 🔧 Dependencies

You’ll likely need to install:

```bash
pip install geopandas shapely pyproj libpysal esda pykrige
```

In some environments (like conda), you may prefer:

```bash
conda install -c conda-forge geopandas libpysal esda pykrige
```


In [ ]:
# ========== 1. Imports & Config (Advanced Geospatial) ==========

from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["figure.dpi"] = 100

# Try geospatial libraries
try:
    import geopandas as gpd
    from shapely.geometry import Point
    GEOPANDAS_AVAILABLE = True
except ImportError:
    GEOPANDAS_AVAILABLE = False
    print("geopandas not installed. Install it to use this template.")

try:
    from libpysal.weights import KNN, Queen
    from esda import Moran, Moran_Local
    PYSAL_AVAILABLE = True
except ImportError:
    PYSAL_AVAILABLE = False
    print("libpysal/esda not installed. Spatial autocorrelation examples will be skipped.")

try:
    from pykrige.ok import OrdinaryKriging
    PYKRIGE_AVAILABLE = True
except ImportError:
    PYKRIGE_AVAILABLE = False
    print("pykrige not installed. Kriging example will be skipped.")

# ---- Config paths ----
DATA_DIR = Path("../input")

POINTS_FILE = "weather_points.csv"   # or .shp, .geojson
POLYGONS_FILE = "counties.shp"      # or .geojson (e.g., county boundaries)

# Core columns
LON_COL = "lon"
LAT_COL = "lat"
VALUE_COL = "temp_high"             # continuous variable for spatial analysis


## 2️⃣ Load Points & Polygons as GeoDataFrames

We assume two main inputs:

1. **Points** – weather observations or stations
   - Columns: `lat`, `lon`, `temp_high`, possibly date
2. **Polygons** – e.g., county or state boundaries

We will:

- Make a `GeoDataFrame` from points
- Ensure everything is in a consistent CRS (coordinate reference system)
- Spatially join points to polygons


In [ ]:
# ========== 2. Load GeoDataFrames ==========

if not GEOPANDAS_AVAILABLE:
    raise ImportError("geopandas is required for this notebook. Install and re-run.")

def load_points(path: Path) -> gpd.GeoDataFrame:
    if path.suffix.lower() in [".shp", ".geojson", ".gpkg"]:
        gdf = gpd.read_file(path)
    else:
        df = pd.read_csv(path)
        if LAT_COL not in df.columns or LON_COL not in df.columns:
            raise ValueError(f"Expected {LAT_COL} and {LON_COL} columns in points CSV.")
        gdf = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(df[LON_COL], df[LAT_COL]),
            crs="EPSG:4326",  # WGS84
        )
    return gdf


def load_polygons(path: Path) -> gpd.GeoDataFrame:
    gdf = gpd.read_file(path)
    return gdf


points_path = DATA_DIR / POINTS_FILE
polygons_path = DATA_DIR / POLYGONS_FILE

points_gdf = load_points(points_path)
polygons_gdf = load_polygons(polygons_path)

print("Points CRS:", points_gdf.crs)
print("Polygons CRS:", polygons_gdf.crs)

# Reproject polygons to match points if needed
if polygons_gdf.crs != points_gdf.crs:
    polygons_gdf = polygons_gdf.to_crs(points_gdf.crs)

print("After alignment - Polygons CRS:", polygons_gdf.crs)

display(points_gdf.head())
display(polygons_gdf.head())


## 3️⃣ Spatial Join: Attach Polygon Attributes to Points

Example use cases:

- Add county/state name to each station point
- Attach socio-economic attributes (population, income, etc.) from polygon layers


In [ ]:
# ========== 3. Spatial Join Points -> Polygons ==========

# Example assumes polygons have a column like 'NAME' for county name; adjust as needed.
poly_cols_to_keep = [c for c in polygons_gdf.columns if c not in ["geometry"]]

points_joined = gpd.sjoin(points_gdf, polygons_gdf[poly_cols_to_keep + ["geometry"]], how="left", predicate="intersects")

print("Joined points shape:", points_joined.shape)
display(points_joined.head())

# Optional: quick plot
ax = polygons_gdf.boundary.plot(color="lightgray", linewidth=0.5)
points_joined.plot(ax=ax, markersize=5, column=VALUE_COL, legend=True)
plt.title("Points colored by value over polygons")
plt.show()


## 4️⃣ Spatial Autocorrelation: Moran's I

We want to check if there is **spatial dependence** in our variable of interest
(e.g., temperature):

- **Moran’s I**: global measure of spatial autocorrelation
- Values:
  - ~0 → random
  - >0 → clustered (similar values near each other)
  - <0 → dispersed (checkerboard-like)


In [ ]:
# ========== 4. Global Moran's I on Points ==========

if not PYSAL_AVAILABLE:
    print("libpysal/esda not available; skipping Moran's I example.")
else:
    # For Moran's I, we need a projected CRS (meters) for distances or KNN with lat/lon.
    # For simplicity, we keep WGS84 and use KNN in "coordinate space".
    # For more rigorous work, reproject to an appropriate projected CRS (e.g., EPSG:3857 or 5070).

    # Build KNN weights (e.g., 8 nearest neighbors)
    w = KNN.from_dataframe(points_joined, k=8)
    w.transform = "R"  # row-standardize

    y = points_joined[VALUE_COL].values
    moran = Moran(y, w)

    print("Moran's I:", moran.I)
    print("p-value (simulated):", moran.p_sim)


## 5️⃣ Local Moran's I (LISA) – Hotspots & Coldspots

**Moran_Local** gives a per-point statistic:

- Highlights clusters of high values (hotspots) and low values (coldspots)
- Can be mapped to see spatial patterns


In [ ]:
# ========== 5. Local Moran (LISA) ==========

if not PYSAL_AVAILABLE:
    print("libpysal/esda not available; skipping Local Moran example.")
else:
    w = KNN.from_dataframe(points_joined, k=8)
    w.transform = "R"

    y = points_joined[VALUE_COL].values
    lisa = Moran_Local(y, w)

    points_joined["local_I"] = lisa.Is
    points_joined["local_p"] = lisa.p_sim

    # Simple quartile classification of local Moran's I
    q = np.quantile(points_joined["local_I"], [0.25, 0.5, 0.75])
    def classify_I(val):
        if val <= q[0]:
            return "Q1"
        elif val <= q[1]:
            return "Q2"
        elif val <= q[2]:
            return "Q3"
        else:
            return "Q4"

    points_joined["local_I_quartile"] = points_joined["local_I"].apply(classify_I)

    ax = polygons_gdf.boundary.plot(color="lightgray", linewidth=0.5)
    points_joined.plot(ax=ax, column="local_I_quartile", categorical=True, legend=True, markersize=8)
    plt.title("Local Moran's I quartiles")
    plt.show()


## 6️⃣ Kriging (Spatial Interpolation) Skeleton

Kriging is a geostatistical method for **interpolating** a variable at unmeasured locations.

Typical use case:

- You have temperature measurements at irregular stations
- You want a smooth temperature field over a grid

Here we show a minimal **Ordinary Kriging** example using `pykrige`.

> ⚠️ This can be computationally heavy on large datasets – consider sampling points.


In [ ]:
# ========== 6. Ordinary Kriging Skeleton ==========

if not PYKRIGE_AVAILABLE:
    print("pykrige not available; skipping kriging example.")
else:
    # Extract coordinates and values
    x = points_joined.geometry.x.values
    y = points_joined.geometry.y.values
    z = points_joined[VALUE_COL].values

    # Build grid for interpolation (in same CRS; if using lat/lon, this is in degrees)
    n_grid = 50  # adjust resolution
    gridx = np.linspace(points_joined.total_bounds[0], points_joined.total_bounds[2], n_grid)
    gridy = np.linspace(points_joined.total_bounds[1], points_joined.total_bounds[3], n_grid)

    OK = OrdinaryKriging(
        x, y, z,
        variogram_model="spherical",  # or "exponential", "gaussian"
        verbose=False,
        enable_plotting=False,
    )

    z_pred, ss = OK.execute("grid", gridx, gridy)

    # Plot kriged surface
    plt.figure(figsize=(8, 6))
    plt.imshow(
        z_pred,
        origin="lower",
        extent=(gridx.min(), gridx.max(), gridy.min(), gridy.max()),
        aspect="auto",
    )
    plt.colorbar(label=VALUE_COL)
    plt.title("Kriged surface")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.show()


## 7️⃣ Using Geospatial Outputs in ML Models

Once you have:

- Joined polygon attributes (e.g., county-level covariates)
- Spatial statistics (e.g., local Moran's I, cluster labels)
- Grid-based interpolated values (e.g., kriged field)

…you can:

- Convert `GeoDataFrame` back to a normal `DataFrame` (keeping numeric columns)
- Train standard ML models (regression/classification) with these **geospatially enriched features**.

Example pattern:

1. Create features: polygon attributes, spatial stats
2. Drop geometry column
3. Use your **tabular regression/classification templates** on the enriched table

This keeps your ML pipeline modular:

- Geospatial notebook(s) handle geometry, spatial joins, kriging, etc.
- Tabular templates handle the rest (EDA, modeling, evaluation).
